In [ ]:
import requests

def check_url_accessibility(url):
    response = requests.head(url)
    if response.status_code == 200:
        print("URL is accessible")
    else:
        print("URL is not accessible")

# URL to check
url = "https://www2.census.gov/programs-surveys/popest/datasets/"
check_url_accessibility(url)

In [ ]:
import os

def download_data(url, filename):
    os.system(f"wget {url} -O {filename}")

# URL and filename
data_url = "https://www2.census.gov/programs-surveys/popest/datasets/"
data_filename = "data.zip"
download_data(data_url, data_filename)

In [ ]:
def upload_to_hdfs(local_path, hdfs_path):
    os.system(f"hadoop fs -put {local_path} {hdfs_path}")

# Paths
local_file_path = "data.zip"
hdfs_file_path = "/user/hadoop/data.zip"
upload_to_hdfs(local_file_path, hdfs_file_path)

In [ ]:
import subprocess

def create_hive_table():
    hive_query = """
    CREATE DATABASE IF NOT EXISTS census_data;
    USE census_data;
    CREATE EXTERNAL TABLE IF NOT EXISTS population_data (
        id INT,
        name STRING,
        value DOUBLE
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    LOCATION '/user/hadoop/data/';
    """
    subprocess.run(["hive", "-e", hive_query])

def load_data_into_hive():
    hive_query = "LOAD DATA INPATH '/user/hadoop/data.zip' INTO TABLE census_data.population_data;"
    subprocess.run(["hive", "-e", hive_query])

create_hive_table()
load_data_into_hive()

In [ ]:
def verify_data():
    query = "SELECT * FROM census_data.population_data LIMIT 10;"
    result = subprocess.run(["hive", "-e", query], capture_output=True, text=True)
    print(result.stdout)

verify_data()